In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from sklearn.model_selection import KFold

In [4]:
train = pd.read_csv("/content/drive/MyDrive/112_1/MachineLearning/train.csv", index_col=0)
test = pd.read_csv("/content/drive/MyDrive/112_1/MachineLearning/test.csv", index_col=0)

In [5]:
train0 = train[train["is_claim"] == 0].sample(1000, random_state=42)
train1 = train[train["is_claim"] == 1].sample(500, random_state=42)
train_new = pd.concat([train0, train1], ignore_index=False)
train_new = train_new.sample(frac=1, random_state=42, ignore_index=False)

In [6]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])


def class_counts(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts


def max_label(dict):
    max_count = 0
    label = ""

    for key, value in dict.items():
        if dict[key] > max_count:
            max_count = dict[key]
            label = key

    return label


def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)


class Question:
    def __init__(self, column, value, header):
        self.column = column
        self.value = value
        self.header = header

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            self.header[self.column], condition, str(self.value))


def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows


def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity


def entropy(rows):
    entries = class_counts(rows)
    avg_entropy = 0
    size = float(len(rows))
    for label in entries:
        prob = entries[label] / size
        avg_entropy = avg_entropy + (prob * math.log(prob, 2))
    return -1*avg_entropy


def information_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))

    return current_uncertainty - p * entropy(left) - (1 - p) * entropy(right)

def find_best_split(rows, header):
    best_gain = 0
    best_question = None
    current_uncertainty = entropy(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val, header)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            gain = information_gain(true_rows, false_rows, current_uncertainty)
            if gain > best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question


class Leaf:
    def __init__(self, rows, id, depth):
        self.predictions = class_counts(rows)
        self.predicted_label = max_label(self.predictions)
        self.id = id
        self.depth = depth


class Decision_Node:
    def __init__(self,
                 question,
                 true_branch,
                 false_branch,
                 depth,
                 id,
                 rows):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.depth = depth
        self.id = id
        self.rows = rows


def build_tree(rows, header, depth=0, id=0):
    gain, question = find_best_split(rows, header)
    if gain == 0:
        return Leaf(rows, id, depth)

    true_rows, false_rows = partition(rows, question)

    true_branch = build_tree(true_rows, header, depth + 1, 2 * id + 2)

    false_branch = build_tree(false_rows, header, depth + 1, 2 * id + 1)

    return Decision_Node(question, true_branch, false_branch, depth, id, rows)


def prune_tree(node, prunedList):
    if isinstance(node, Leaf):
        return node

    if int(node.id) in prunedList:
        return Leaf(node.rows, node.id, node.depth)

    node.true_branch = prune_tree(node.true_branch, prunedList)

    node.false_branch = prune_tree(node.false_branch, prunedList)

    return node


def classify(row, node):
    if isinstance(node, Leaf):
        return node.predicted_label
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print(spacing + "Leaf id: " + str(node.id) + " Predictions: " + str(node.predictions) + " Label Class: " + str(node.predicted_label))
        return

    print(spacing + str(node.question) + " id: " + str(node.id) + " depth: " + str(node.depth))

    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")


def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs


def getLeafNodes(node, leafNodes =[]):
    if isinstance(node, Leaf):
        leafNodes.append(node)
        return

    getLeafNodes(node.true_branch, leafNodes)

    getLeafNodes(node.false_branch, leafNodes)

    return leafNodes


def getInnerNodes(node, innerNodes =[]):

    if isinstance(node, Leaf):
        return

    innerNodes.append(node)

    getInnerNodes(node.true_branch, innerNodes)

    getInnerNodes(node.false_branch, innerNodes)

    return innerNodes


def computeAccuracy(rows, node):

    count = len(rows)
    if count == 0:
        return 0

    accuracy = 0
    for row in rows:
        if row[-1] == classify(row, node):
            accuracy += 1
    return round(accuracy/count, 2)

In [ ]:
k1 = 3
CV1 = KFold(n_splits=k1, shuffle=False, random_state=None)
for train_index, valid_index in CV1.split(train_new):
  print("Train data", train_index,"Valid data", valid_index)
  header = list(train_new.columns)
  lst = train.values.tolist()

  CV1_train, CV1_valid = train_new.iloc[train_index], train_new.iloc[valid_index]
  CV1_train = CV1_train.to_numpy()
  CV1_valid = CV1_valid.to_numpy()

  t = build_tree(CV1_train, header)

  print("\nLeaf nodes ****************")
  leaves = getLeafNodes(t)
  #for leaf in leaves:
   # print("id = " + str(leaf.id) + " depth =" + str(leaf.depth))

  print("\nNon-leaf nodes ****************")
  innerNodes = getInnerNodes(t)

  #for inner in innerNodes:
   # print("id = " + str(inner.id) + " depth =" + str(inner.depth))

  beforeAccuracy = maxAccuracy = computeAccuracy(CV1_valid, t)
  print("\nTree before pruning with accuracy: " + str(maxAccuracy*100) + "\n")
  #print_tree(t)

  nodeIdToPrune = -1
  for node in innerNodes:
    if node.id != 0:
        prune_tree(t, [node.id])
        currentAccuracy = computeAccuracy(CV1_valid, t)
        print("Pruned node_id: " + str(node.id) + " to achieve accuracy: " + str(currentAccuracy*100) + "%")
        if currentAccuracy > maxAccuracy:
            maxAccuracy = currentAccuracy
            nodeIdToPrune = node.id
        t = build_tree(CV1_train, header)
        if maxAccuracy == 1:
            break
  print("Before", beforeAccuracy)

  nodeIdToPrune = 1
  if nodeIdToPrune != -1:
    t = build_tree(CV1_train, header)
    prune_tree(t, [nodeIdToPrune])
    print("\nFinal node Id to prune (for max accuracy): " + str(nodeIdToPrune))
  else:
    t = build_tree(CV1_train, header)
    print("\nPruning strategy didn't increased accuracy")

  print("\n********************************************************************")
  print("*********** Final Tree with accuracy: " + str(maxAccuracy*100) + "%  ************")
  print("********************************************************************\n")
  #print_tree(t)

Train data [ 500  501  502  503  504  505  506  507  508  509  510  511  512  513
  514  515  516  517  518  519  520  521  522  523  524  525  526  527
  528  529  530  531  532  533  534  535  536  537  538  539  540  541
  542  543  544  545  546  547  548  549  550  551  552  553  554  555
  556  557  558  559  560  561  562  563  564  565  566  567  568  569
  570  571  572  573  574  575  576  577  578  579  580  581  582  583
  584  585  586  587  588  589  590  591  592  593  594  595  596  597
  598  599  600  601  602  603  604  605  606  607  608  609  610  611
  612  613  614  615  616  617  618  619  620  621  622  623  624  625
  626  627  628  629  630  631  632  633  634  635  636  637  638  639
  640  641  642  643  644  645  646  647  648  649  650  651  652  653
  654  655  656  657  658  659  660  661  662  663  664  665  666  667
  668  669  670  671  672  673  674  675  676  677  678  679  680  681
  682  683  684  685  686  687  688  689  690  691  692  693  694 

1. Tree before pruning with accuracy: 62.0
   
   Final Tree with accuracy: 65.0%

2. Tree before pruning with accuracy: 61.0
   
   Final Tree with accuracy: 69.0%

3. Tree before pruning with accuracy: 59.0

   Final Tree with accuracy: 64.0%

In [6]:
before = []
after = []
k2 = 5
CV2 = KFold(n_splits=k2)
for train_index, valid_index in CV2.split(train_new):
    header = list(train_new.columns)
    lst = train_new.values.tolist()

    CV2_train, CV2_valid = train_new.iloc[train_index], train.iloc[valid_index]
    CV2_train = CV2_train.to_numpy()
    CV2_valid = CV2_valid.to_numpy()

    t = build_tree(CV2_train, header)

    print("\nLeaf nodes ****************")
    leaves = getLeafNodes(t)
    #for leaf in leaves:
     # print("id = " + str(leaf.id) + " depth =" + str(leaf.depth))

    print("\nNon-leaf nodes ****************")
    innerNodes = getInnerNodes(t)

    #for inner in innerNodes:
     # print("id = " + str(inner.id) + " depth =" + str(inner.depth))

    beforeAccuracy = maxAccuracy = computeAccuracy(CV2_valid, t)
    print("\nTree before pruning with accuracy: " + str(maxAccuracy*100) + "\n")
    #print_tree(t)

    nodeIdToPrune = -1
    for node in innerNodes:
      if node.id != 0:
        prune_tree(t, [node.id])
        currentAccuracy = computeAccuracy(CV2_valid, t)
        print("Pruned node_id: " + str(node.id) + " to achieve accuracy: " + str(currentAccuracy*100) + "%")
        if currentAccuracy > maxAccuracy:
            maxAccuracy = currentAccuracy
            nodeIdToPrune = node.id
        t = build_tree(CV2_train, header)
        if maxAccuracy == 1:
            break
    print("Before", beforeAccuracy)

    nodeIdToPrune = 1
    if nodeIdToPrune != -1:
      t = build_tree(CV2_train, header)
      prune_tree(t, [nodeIdToPrune])
      print("\nFinal node Id to prune (for max accuracy): " + str(nodeIdToPrune))
    else:
      t = build_tree(CV2_train, header)
      print("\nPruning strategy didn't increased accuracy")

    print("\n********************************************************************")
    print("*********** Final Tree with accuracy: " + str(maxAccuracy*100) + "%  ************")
    print("********************************************************************\n")
    before.append(beforeAccuracy)
    after.append(maxAccuracy)
    #print_tree(t)

Streaming output truncated to the last 5000 lines.
Pruned node_id: 26984482 to achieve accuracy: 69.0%
Pruned node_id: 53968965 to achieve accuracy: 69.0%
Pruned node_id: 107937931 to achieve accuracy: 69.0%
Pruned node_id: 215875863 to achieve accuracy: 69.0%
Pruned node_id: 431751728 to achieve accuracy: 69.0%
Pruned node_id: 431751727 to achieve accuracy: 69.0%
Pruned node_id: 863503456 to achieve accuracy: 69.0%
Pruned node_id: 1727006914 to achieve accuracy: 69.0%
Pruned node_id: 863503455 to achieve accuracy: 69.0%
Pruned node_id: 1727006912 to achieve accuracy: 69.0%
Pruned node_id: 1645 to achieve accuracy: 69.0%
Pruned node_id: 3292 to achieve accuracy: 69.0%
Pruned node_id: 6585 to achieve accuracy: 69.0%
Pruned node_id: 13172 to achieve accuracy: 69.0%
Pruned node_id: 203 to achieve accuracy: 74.0%
Pruned node_id: 408 to achieve accuracy: 70.0%
Pruned node_id: 818 to achieve accuracy: 70.0%
Pruned node_id: 1637 to achieve accuracy: 70.0%
Pruned node_id: 3275 to achieve accur

In [7]:
print(before)
print(after)

[0.69, 0.7, 0.67, 0.7, 0.67]
[0.86, 0.89, 0.88, 0.91, 0.81]


In [ ]:
before = []
after = []

k3 = 10
CV3 = KFold(n_splits=k3)
for train_index, valid_index in CV3.split(train_new):
    header = list(train_new.columns)
    lst = train.values.tolist()

    CV3_train, CV3_valid = train_new.iloc[train_index], train.iloc[valid_index]
    CV3_train = CV3_train.to_numpy()
    CV3_valid = CV3_valid.to_numpy()

    t = build_tree(CV3_train, header)

    print("\nLeaf nodes ****************")
    leaves = getLeafNodes(t)
    #for leaf in leaves:
     # print("id = " + str(leaf.id) + " depth =" + str(leaf.depth))

    print("\nNon-leaf nodes ****************")
    innerNodes = getInnerNodes(t)

    #for inner in innerNodes:
     # print("id = " + str(inner.id) + " depth =" + str(inner.depth))

    beforeAccuracy = maxAccuracy = computeAccuracy(CV3_valid, t)
    print("\nTree before pruning with accuracy: " + str(maxAccuracy*100) + "\n")
    #print_tree(t)

    nodeIdToPrune = -1
    for node in innerNodes:
      if node.id != 0:
        prune_tree(t, [node.id])
        currentAccuracy = computeAccuracy(CV3_valid, t)
        print("Pruned node_id: " + str(node.id) + " to achieve accuracy: " + str(currentAccuracy*100) + "%")
        if currentAccuracy > maxAccuracy:
            maxAccuracy = currentAccuracy
            nodeIdToPrune = node.id
        t = build_tree(CV3_train, header)
        if maxAccuracy == 1:
            break
    print("Before", beforeAccuracy)

    nodeIdToPrune = 1
    if nodeIdToPrune != -1:
      t = build_tree(CV3_train, header)
      prune_tree(t, [nodeIdToPrune])
      print("\nFinal node Id to prune (for max accuracy): " + str(nodeIdToPrune))
    else:
      t = build_tree(CV3_train, header)
      print("\nPruning strategy didn't increased accuracy")

    print("\n********************************************************************")
    print("*********** Final Tree with accuracy: " + str(maxAccuracy*100) + "%  ************")
    print("********************************************************************\n")

    before.append(beforeAccuracy)
    after.append(maxAccuracy)
    #print_tree(t)


Leaf nodes ****************

Non-leaf nodes ****************

Tree before pruning with accuracy: 69.0

Pruned node_id: 2 to achieve accuracy: 86.0%
Pruned node_id: 6 to achieve accuracy: 77.0%
Pruned node_id: 14 to achieve accuracy: 68.0%
Pruned node_id: 30 to achieve accuracy: 68.0%
Pruned node_id: 62 to achieve accuracy: 68.0%
Pruned node_id: 126 to achieve accuracy: 68.0%
Pruned node_id: 254 to achieve accuracy: 69.0%
Pruned node_id: 13 to achieve accuracy: 77.0%
Pruned node_id: 27 to achieve accuracy: 76.0%
Pruned node_id: 56 to achieve accuracy: 69.0%
Pruned node_id: 55 to achieve accuracy: 76.0%
Pruned node_id: 112 to achieve accuracy: 71.0%
Pruned node_id: 226 to achieve accuracy: 71.0%
Pruned node_id: 454 to achieve accuracy: 69.0%
Pruned node_id: 910 to achieve accuracy: 69.0%
Pruned node_id: 1822 to achieve accuracy: 69.0%
Pruned node_id: 3645 to achieve accuracy: 69.0%
Pruned node_id: 7291 to achieve accuracy: 69.0%
Pruned node_id: 14584 to achieve accuracy: 69.0%
Pruned no

In [ ]:
print(before)
print(after)